In [8]:
import os
import requests
import json
import csv
from os import listdir
from os.path import isfile, join
import zipfile
import pandas as pd

In [ ]:
def download_cve_main_zip():
    url = 'https://github.com/CVEProject/cvelistV5/archive/refs/heads/main.zip'
    filePath = "cveData"
    filename = "main.zip"

    # Check if directory exists; if not, create it
    if not os.path.exists(filePath):
        os.makedirs(filePath)

    print(f"Downloading {filename}...")

    # Download the file with streaming
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(f"{filePath}/{filename}", 'wb') as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)
        print(f"Downloaded {filename} successfully.")
    else:
        print(f"Failed to download {filename}. Status code: {response.status_code}")

download_cve_main_zip()

In [ ]:

def unzip_data():
    # Define source path for zip files and destination for extraction
    zip_folder = "cveData"
    extract_folder = "jsonFile2"

    # Ensure the extraction directory exists
    if not os.path.exists(extract_folder):
        os.makedirs(extract_folder)

    #Get a list of all files in the zip folder
    files = [f for f in listdir(zip_folder) if isfile(join(zip_folder, f))]
    files.sort()

    for file in files:
        if file.endswith(".zip"):
            print(f"Opening: {file}")
            with zipfile.ZipFile(join(zip_folder, file), 'r') as archive:
                archive.extractall(extract_folder)
                print(f"Extracted {file} successfully.")

unzip_data()

In [2]:
def generate_CVSSV3csv_for_training():
# Traverse the jsonFile2 directory and its subdirectories
    json_files_path = "jsonFile2/cvelistV5-main/cves"
    fileName = 'Vulnerabilities_Details.csv'

    with open(fileName, 'w', newline='') as f_output:
        csv_output = csv.writer(f_output)
        csv_output.writerow(['Vulnerability Type', 'Vulnerability Title', 'Description'])

        for root, _, files in os.walk(json_files_path):
            for file in files:
                if file.endswith(".json"):
                    with open(join(root, file), 'r') as json_file:
                        try:
                            cve_data = json.load(json_file)

                            if 'cveMetadata' in cve_data and 'containers' in cve_data:
                                containers = cve_data['containers']
                                if 'cna' in containers:
                                    container = containers['cna']

                                    title = container.get('title', 'N/A')
                                    descriptions = container.get('descriptions', [])

                                    if descriptions:
                                        description = descriptions[0].get('value', 'N/A')
                                    else:
                                        description = 'N/A'

                                    problem_types = container.get('problemTypes', [])

                                    vuln_type = 'N/A'
                                    if problem_types:
                                        descriptions = problem_types[0].get('descriptions', [])
                                        if descriptions:
                                            vuln_type = descriptions[0].get('description', 'N/A')

                                    csv_output.writerow([vuln_type, title, description])

                        except json.JSONDecodeError:
                            print(f"Error decoding JSON from file: {file}")
                        except KeyError as e:
                            print(f"Missing key {e} in file: {file}")

# Call the functions
generate_CVSSV3csv_for_training() # 0

In [ ]:
df = pd.read_csv('Vulnerabilities_Details.csv')

# Filter out rows where 'Vulnerability Type' is 'N/A', 'n/a', or NaN
filtered_df = df[df['Vulnerability Type'].str.lower().notnull() & (df['Vulnerability Type'].str.lower() != 'n/a')]

# Display the first few rows of the filtered DataFrame
print(filtered_df.head())

# If you want to save the filtered DataFrame back to a CSV file
filtered_df.to_csv('Filtered_Vulnerabilities_Details.csv', index=False)
